In [3]:
import os
import cv2
import numpy as np
import glob
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

def draw_fragments_2d(stmap_labels):
    m,n = stmap_labels.shape
    ids = np.unique(stmap_labels)
    size = len(ids)
#     print("the number of instance is %d" % size)
    color_pred = np.zeros([m, n, 3], dtype=np.uint8)
    idx = np.searchsorted(ids, stmap_labels)
    for i in range(3):
        color_val = np.random.randint(0, 255, ids.shape)
        if ids[0] == 0:
            color_val[0] = 0
        color_pred[:,:,i] = color_val[idx]
    return color_pred 


def remove_small(binary_im, min_size = 50):
    
    # find all of the connected components (white blobs in your image).
    # im_with_separated_blobs is an image where each detected blob has a different pixel value ranging from 1 to nb_blobs - 1.
    nb_blobs, im_with_separated_blobs, stats, _ = cv2.connectedComponentsWithStats(binary_im)
    # stats (and the silenced output centroids) gives some information about the blobs. See the docs for more information. 
    # here, we're interested only in the size of the blobs, contained in the last column of stats.
    sizes = stats[:, -1]
    # the following lines result in taking out the background which is also considered a component, which I find for most applications to not be the expected output.
    # you may also keep the results as they are by commenting out the following lines. You'll have to update the ranges in the for loop below. 
    sizes = sizes[1:]
    nb_blobs -= 1

    # output image with only the kept components
    im_result = np.zeros((binary_im.shape))
    # for every component in the image, keep it only if it's above min_size
    for blob in range(nb_blobs):
        if sizes[blob] >= min_size:
            # see description of im_with_separated_blobs above
            im_result[im_with_separated_blobs == blob + 1] = 255
            
    return im_result.astype(np.uint8)

offset = [7, 22]

four_image_size = [1052, 1112, 3]

In [12]:
# ResUnet-ST

img_folder = "ResUnet/test/affs_model-006000/affs_img/"
binary_mask_folder = "ResUnet/test/affs_model-006000/Binary"
instance_mask_folder = "ResUnet/test/affs_model-006000/Instance"

if not os.path.exists(binary_mask_folder):
    os.makedirs(binary_mask_folder)
    
if not os.path.exists(instance_mask_folder):
    os.makedirs(instance_mask_folder)

sorted_image_files = sorted(glob.glob(os.path.join(img_folder, '*.png')))


for image_path in sorted_image_files:
    
    image_basename = os.path.basename(image_path)
    
    four_images = cv2.imread(image_path)
    
    binary_img = four_images[512 + 7*3: 1052 - 7, 512 + 22*3 : 1112 - 22*1]
    
    binary_img = cv2.cvtColor(binary_img, cv2.COLOR_BGR2GRAY)
    
    binary_mask = remove_small(binary_img, min_size = 50)

    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(binary_mask, connectivity=8)

    color_mask = draw_fragments_2d(output)
    
    mask_filename = os.path.join(instance_mask_folder, image_basename) 
    fg_filename = os.path.join(binary_mask_folder, image_basename)

    cv2.imwrite(mask_filename, color_mask)
    cv2.imwrite(fg_filename, binary_mask)
    

the number of instance is 26
the number of instance is 31
the number of instance is 30
the number of instance is 36
the number of instance is 34
the number of instance is 24
the number of instance is 30
the number of instance is 29
the number of instance is 27
the number of instance is 20
the number of instance is 36
the number of instance is 26
the number of instance is 26
the number of instance is 29
the number of instance is 35
the number of instance is 28
the number of instance is 31
the number of instance is 30
the number of instance is 24
the number of instance is 33
the number of instance is 32
the number of instance is 25
the number of instance is 23
the number of instance is 28
the number of instance is 28
the number of instance is 29
the number of instance is 29
the number of instance is 39
the number of instance is 31
the number of instance is 33
the number of instance is 25
the number of instance is 29
the number of instance is 30
the number of instance is 18
the number of 

In [13]:
# AttUnet-ST

img_folder = "AttentionUnet/test/affs_model-020000/affs_img"

img_folder = "ResUnet/test/affs_model-006000/affs_img/"
binary_mask_folder = "AttentionUnet/test/affs_model-020000/Binary"
instance_mask_folder = "AttentionUnet/test/affs_model-020000/Instance"

if not os.path.exists(binary_mask_folder):
    os.makedirs(binary_mask_folder)
    
if not os.path.exists(instance_mask_folder):
    os.makedirs(instance_mask_folder)

sorted_image_files = sorted(glob.glob(os.path.join(img_folder, '*.png')))


for image_path in sorted_image_files:
    
    image_basename = os.path.basename(image_path)
    
    four_images = cv2.imread(image_path)
    
    binary_img = four_images[512 + 7*3: 1052 - 7*1, 512 + 22*3 : 1112 - 22*1]
    
    binary_img = cv2.cvtColor(binary_img, cv2.COLOR_BGR2GRAY)
    
    binary_mask = remove_small(binary_img, min_size = 50)

    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(binary_mask, connectivity=8)

    color_mask = draw_fragments_2d(output)
    
    mask_filename = os.path.join(instance_mask_folder, image_basename) 
    fg_filename = os.path.join(binary_mask_folder, image_basename)

    cv2.imwrite(mask_filename, color_mask)
    cv2.imwrite(fg_filename, binary_mask)


the number of instance is 26
the number of instance is 31
the number of instance is 30
the number of instance is 36
the number of instance is 34
the number of instance is 24
the number of instance is 30
the number of instance is 29
the number of instance is 27
the number of instance is 20
the number of instance is 36
the number of instance is 26
the number of instance is 26
the number of instance is 29
the number of instance is 35
the number of instance is 28
the number of instance is 31
the number of instance is 30
the number of instance is 24
the number of instance is 33
the number of instance is 32
the number of instance is 25
the number of instance is 23
the number of instance is 28
the number of instance is 28
the number of instance is 29
the number of instance is 29
the number of instance is 39
the number of instance is 31
the number of instance is 33
the number of instance is 25
the number of instance is 29
the number of instance is 30
the number of instance is 18
the number of 